# [Getting Started With TensorFlow](https://www.tensorflow.org/get_started/get_started)

In [1]:
import tensorflow as tf
import numpy as np
logDir = 'logs/getting started'

In [2]:
tf.__version__

'1.0.1'

## Constants

In [3]:
a = tf.constant(3)
b = tf.constant(4)
c = a + b

with tf.Session() as sess:
    (aRes, bRes, cRes) = sess.run([a, b, c])
    print(aRes, bRes, cRes)
    tf.summary.FileWriter(logDir, sess.graph)

3 4 7


## Placeholders

In [4]:
a = tf.placeholder(dtype=tf.float64, shape=[2, 4])
b = tf.placeholder(dtype=tf.float64, shape=[4, 1])
c = tf.matmul(a, b)

with tf.Session() as sess:
    aVal = [[3, 5, 8, 1], [2, 9, 4, 2]]
    bVal = [[4], [8], [0], [3]]
    (aRes, bRes, cRes) = sess.run((a, b, c), feed_dict={a: aVal, b:bVal})
    print(aRes)
    print(bRes)
    print(cRes)
    tf.summary.FileWriter(logDir, sess.graph)

[[ 3.  5.  8.  1.]
 [ 2.  9.  4.  2.]]
[[ 4.]
 [ 8.]
 [ 0.]
 [ 3.]]
[[ 55.]
 [ 86.]]


## Variables

In [5]:
def statistics(a, b, c):
    randomize = tf.assign(a, tf.random_normal(dtype=tf.float64, shape=[4]))
    (mean, stdev) = tf.nn.moments(a, axes=[0])
    setMean = tf.assign(b, mean)
    setStdev = tf.assign(c, stdev)
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        sess.run(randomize)
        sess.run((setMean, setStdev))
        (aRes, bRes, cRes) = sess.run((a, b, c))
        print(aRes)
        print(bRes)
        print(cRes)
        tf.summary.FileWriter(logDir, sess.graph)
        
    
a = tf.Variable([0, 0, 0, 0], dtype=tf.float64)
b = tf.Variable(0, dtype=tf.float64)
c = tf.Variable(0, dtype=tf.float64)
statistics(a, b, c)

[-0.60279095  1.18525495 -0.04886636 -1.91084952]
-0.344312970718
1.23692858716


## ``tf.train``

In [23]:
nSamples = 1000
nFeatures = 3
nTrain = int(nSamples * 0.8)

X = tf.placeholder(dtype=tf.float32, name='X')
y = tf.placeholder(dtype=tf.float32, name='y')
a = tf.Variable([[0], [0], [0]], dtype=tf.float32, name='a')
b = tf.Variable(0, dtype=tf.float32, name='b')

prediction = tf.matmul(X, a) + b
cost = tf.reduce_mean(tf.square(prediction - y))
optimizer = tf.train.GradientDescentOptimizer(0.003)
train = optimizer.minimize(cost)
init = tf.global_variables_initializer()

In [24]:
XSamples = np.random.randn(nSamples, nFeatures) * 10 + 3
def calculateY(x):
    return [x[0] * 4.2 + x[1] * (-2.2) + x[2] * 0.48 + 3.8 - np.random.normal(loc=2.22, scale=0.25)]

ySamples = np.apply_along_axis(arr=XSamples, func1d=calculateY, axis=1)

X_train = XSamples[: nTrain, :]
y_train = ySamples[: nTrain, :]
X_test = XSamples[nTrain :, :]
y_test = ySamples[nTrain :, :]

In [8]:
print(X_train.shape)
X_train[: 6, :]

(800, 3)


array([[ 12.20124728,   2.5398655 ,   8.80046104],
       [ -8.84608428,  -2.70371496,  -1.61742939],
       [ -1.09668924,  -6.99596994,  15.59148976],
       [ 14.34560048,   0.95803639,  25.70520824],
       [ 11.82515693,   1.40197366,   9.54464551],
       [  3.24737932,  26.00698478,   0.94306032]])

In [9]:
print(X_test.shape)
X_test[: 6, :]

(200, 3)


array([[ -3.30252904e-01,   1.50951787e+01,   6.66171163e+00],
       [  1.29261772e+01,  -5.22915627e+00,   1.75905124e+01],
       [ -1.43550430e+01,   1.42803789e+01,   2.20685069e+00],
       [ -6.76525268e+00,   1.08565869e+01,   1.99767552e-02],
       [  6.65704631e+00,  -1.03037930e+01,   2.09280275e+00],
       [ -1.06531677e+01,   2.75097098e+01,   1.71980115e+00]])

In [10]:
print(y_train.shape)
y_train[: 6]

(800, 1)


array([[ 51.80962364],
       [-30.21258113],
       [ 19.67206929],
       [ 72.18222245],
       [ 52.39921053],
       [-41.34869118]])

In [11]:
print(y_test.shape)
y_test[: 6]

(200, 1)


array([[ -29.75754717],
       [  75.48195547],
       [ -89.08376795],
       [ -50.5912982 ],
       [  53.42229165],
       [-102.83625617]])

In [12]:
with tf.Session() as sess:
    sess.run(init)
    tf.summary.FileWriter(logDir, sess.graph)
    feed_train = {X: X_train, y: y_train}
    
    for _ in range(10000):
        sess.run(train, feed_dict=feed_train)

    (aRes, bRes, costRes) = sess.run((a, b, cost), feed_dict=feed_train)
    print('aRes: {0}'.format(aRes))
    print('bRes: {0}'.format(bRes))
    print('costRes: {0}'.format(costRes))
    
    feed_test = {X: X_test, y: y_test}
    
    y_predict = sess.run(prediction, feed_dict=feed_test)
    print('y_predict: {0}'.format(y_predict[: 6]))
    print('y_test: {0}'.format(y_test[: 6]))
    cost_test = sess.run(cost, feed_dict=feed_test)
    print('cost_test: {0}'.format(cost_test))

aRes: [[ 4.20036316]
 [-2.20034289]
 [ 0.48015046]]
bRes: 1.5813637971878052
costRes: 0.05771639943122864
y_predict: [[ -29.8217659 ]
 [  75.82803345]
 [ -89.07713318]
 [ -50.71377563]
 [  53.22011185]
 [-102.87084198]]
y_test: [[ -29.75754717]
 [  75.48195547]
 [ -89.08376795]
 [ -50.5912982 ]
 [  53.42229165]
 [-102.83625617]]
cost_test: 0.06520393490791321


## ``tf.contrib.learn``

### Basic usage

In [13]:
features = [tf.contrib.layers.real_valued_column(column_name='X', dimension=3)]
trainer = tf.contrib.learn.LinearRegressor(feature_columns=features, model_dir=logDir)
input_fn_train = tf.contrib.learn.io.numpy_input_fn({'X': X_train}, y_train, batch_size=200, num_epochs=800)
trainer.fit(input_fn=input_fn_train, steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_environment': 'local', '_evaluation_master': '', '_task_id': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_master': '', '_tf_random_seed': None, '_save_checkpoints_steps': None, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10ec2bda0>, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_is_chief': True}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 154201 into logs/model.ckpt.
I

LinearRegressor(params={'joint_weights': False, 'optimizer': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x10eae3f28>, 'gradient_clip_norm': None, 'feature_columns': [_RealValuedColumn(column_name='X', dimension=3, default_value=None, dtype=tf.float32, normalizer=None)]})

In [18]:
print('a: {0}'.format(trainer.get_variable_value('linear/X/weight')))
print('b: {0}'.format(trainer.get_variable_value('linear/bias_weight')))

def input_test():
    return {
        'X': X_test,
        'y': y_test
    }

y_predict = list(trainer.predict(input_fn=input_test))

input_fn_eval = tf.contrib.learn.io.numpy_input_fn(x={'X': X_test}, y=y_test)
evalRes = trainer.evaluate(input_fn=input_fn_eval)
print('y_predict: {0}'.format(y_predict[: 6]))
print('y_test: {0}'.format(y_test.reshape(-1,)[: 6]))
print('evalRes: {0}'.format(evalRes))

a: [[ 4.2003603 ]
 [-2.20035529]
 [ 0.48015895]]
b: [ 1.58134663]
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-04-11-14:09:24
INFO:tensorflow:Finished evaluation at 2017-04-11-14:09:24
INFO:tensorflow:Saving dict for global step 157400: global_step = 157400, loss = 0.0462997
y_predict: [-29.821909, 75.828194, -89.077278, -50.713909, 53.220222, -102.87115]
y_test: [ -29.75754717   75.48195547  -89.08376795  -50.5912982    53.42229165
 -102.83625617]
evalRes: {'global_step': 157400, 'loss': 0.046299677}


### Custom model

In [28]:
def model(features, y, mode):
    a = tf.get_variable(name='a', shape=[3, 1], dtype=tf.float64)
    b = tf.get_variable(name='b', shape=1, dtype=tf.float64)
    X = features['X']
    predictions = tf.matmul(X, a) + b
    loss = tf.reduce_mean(tf.square(predictions - y))
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.003)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    
    return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# X_train = np.array([[2.2, 3.3, 4.4], [1.5, 4.6, 7.8]])
# y_train = np.array([[9], [15]])
input_fn_train = tf.contrib.learn.io.numpy_input_fn(x={'X': X_train}, y=y_train, batch_size=200, num_epochs=1000)
estimator.fit(input_fn=input_fn_train, steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_environment': 'local', '_evaluation_master': '', '_task_id': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_master': '', '_tf_random_seed': None, '_save_checkpoints_steps': None, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114408c18>, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_is_chief': True}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/bs/_rnbrf4j6918fhbsgh06x6q40000gp/T/tmp4qd1kosc/model.ckpt.
INFO:tensorflow:loss = 2173.09889065, step = 1
INFO:tensorflow:global_step/sec: 997.516
INFO:tensorflow:loss = 0.155520159832, step = 101
INFO:tensorflow:global_step/sec: 1062.58
INFO:tensorflow:loss = 0.11463080279, step = 201
INFO:tensorflow:global_step/sec: 1092.35
INFO:tenso

Estimator(params=None)

In [35]:
print('a: {0}'.format(estimator.get_variable_value('a')))
print('b: {0}'.format(estimator.get_variable_value('b')))

def input_test():
    return {
        'X': X_test,
        'y': y_test
    }

y_predict = [item[0] for item in estimator.predict(input_fn=input_test)]

input_fn_eval = tf.contrib.learn.io.numpy_input_fn(x={'X': X_test}, y=y_test)
evalRes = estimator.evaluate(input_fn=input_fn_eval)
print('y_predict: {0}'.format(y_predict[: 6]))
print('y_test: {0}'.format(y_test.reshape(-1,)[: 6]))
print('evalRes: {0}'.format(evalRes))

a: [[ 4.20239728]
 [-2.19961867]
 [ 0.47976918]]
b: [ 1.58433718]
INFO:tensorflow:Starting evaluation at 2017-04-11-14:20:04
INFO:tensorflow:Finished evaluation at 2017-04-11-14:20:04
INFO:tensorflow:Saving dict for global step 4001: global_step = 4001, loss = 0.0389435
y_predict: [11.210544521621516, -40.679471594842639, 21.659016263188487, 12.62510152320772, 36.564740182826263, 69.104542108821164]
y_test: [ 11.22785878 -40.68100867  21.78221009  12.92040124  36.37709252
  68.91257175]
evalRes: {'global_step': 4001, 'loss': 0.038943511}


In [ ]:
tf.contrib.learn.ModelFnOps?

## Using Tensorboard

Run this command on terminal
```shell
$ tensorboard --logdir=logs
```

In [ ]:
!open http://localhost:6006